In [110]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
import numpy as np
from datasets import load_metric
import evaluate
import accelerate

In [111]:
data = load_dataset("dair-ai/emotion")

c:\Users\furkanbaba\Desktop\furkanbaba\coding\sentiment-analysis\venv\lib\site-packages\datasets\load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [112]:
data["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [113]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = data.map(tokenize_function, batched=True,
                              batch_size=None)


In [114]:
tokenized_datasets["train"].column_names

['text', 'label', 'input_ids', 'attention_mask']

In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [116]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, 
                           references = labels)

In [117]:
import transformers

import torch

print("transformers version:", transformers.__version__)
print("accelerate version:", accelerate.__version__)
print("torch version:", torch.__version__)


transformers version: 4.36.2
accelerate version: 0.25.0
torch version: 2.1.2+cpu


In [118]:
sample_text = "I love natural language processing!"